# 0. Install and Import Dependencies

In [1]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install nbimporter
!pip install nbformat
!pip install pygltflib
!pip install trimesh
!pip install pyrender


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import time
import cv2
import numpy as np
import math
import os
from collections import deque
import logging
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
import math
import trimesh
import pyrender
from pygltflib import GLTF2
from scipy.spatial.transform import Rotation as R

# gltf = GLTF2().load("3D\magicmirror.glb")
# Load the glTF file
def get_joint_transform(gltf,joint_index):
    joint_node = gltf.nodes[joint_index]

    # Use default values if translation, rotation, or scale are not present
    translation = np.array(joint_node.translation or [0, 0, 0])
    rotation = R.from_quat(joint_node.rotation or [0, 0, 0, 1]).as_matrix()
    scale = np.array(joint_node.scale or [1, 1, 1])

    # Build the 4x4 transformation matrix directly
    transform = np.eye(4)
    transform[:3, :3] = rotation * scale
    transform[:3, 3] = translation

    return transform

# Load the GLB file 
gltf = GLTF2().load("3D/magicmirror.glb")
m_index = 0
material = gltf.materials[m_index]
material.alphaMode = "MASK"
material.alphaCutoff=0.5
for node_idx, node in enumerate(gltf.nodes):
    if node.skin is not None:
        joints = gltf.skins[node.skin].joints
        bone_list=[]
        for joint in joints:
            joint_node = gltf.nodes[joint]
            bone_list.append(joint_node)
    if node.children:
        node.children = [child for child in node.children if child != 9 ]
for bone in bone_list:
    print(f"bone {bone.name} rotation:{bone.rotation} and translation:{bone.translation}")
if 9 not in gltf.scenes[0].nodes:
    gltf.scenes[0].nodes.append(9)
    print("Not in list")
else:
    print("In list")
#multiply bone transformations to mesh
# Access the first mesh in the scene (or iterate if you have more meshes)
mesh = gltf.meshes[0]
for primitive in mesh.primitives:
    position = gltf.accessors[primitive.attributes.POSITION]
    print(f"Position {position}")
gltf.save("3D\magicmirror.glb")

# Create a Pyrender scene
s = pyrender.Scene()
mesh_vertices = []
# Convert trimesh scene into pyrender meshes and add them
trimesh_scene = trimesh.load('3D/magicmirror.glb')
for name, mesh in trimesh_scene.geometry.items():
    pyrender_mesh = pyrender.Mesh.from_trimesh(mesh)
    node = pyrender.Node(mesh=pyrender_mesh)
    print("node")
    s.add_node(node)
    mesh_vertices= mesh.vertices
print("length",mesh_vertices[0])
mesh = gltf.meshes[0]
vBind = np.vstack(mesh_vertices).astype(np.float32)

# Make vBind read-only to prevent modification
vBind.setflags(write=False)
def apply_skinning(gltf, vertices, joints, weights, inverse_bind_matrices):
    transformed_vertices = np.zeros_like(mesh_vertices)  # Allocate transformed vertex positions
    
    for i, v in enumerate(vertices):
        weighted_sum = np.zeros(3)  # Accumulate weighted transformation
        for j in range(4):  # Each vertex is influenced by up to 4 joints
            joint_index = joints[i][j]  # Get joint affecting this vertex
            weight = weights[i][j]  # Get corresponding weight

            # Get transformation matrix for this joint
            bind_matrix = inverse_bind_matrices[joint_index]
            joint_transform = get_joint_transform(gltf, joint_index)

            # Transform vertex using bind pose and skinning equation
            transformed_joint = (bind_matrix @ np.append(v, 1))[:3]  # Apply bind pose
            skinned_position = (joint_transform @ np.append(transformed_joint, 1))[:3]  # Apply joint transform

            # Weighted sum of transformations
            weighted_sum += weight * skinned_position

        transformed_vertices[i] = weighted_sum  # Update vertex position
    
    return transformed_vertices
# Get joint weights and indices from GLTF
def active_updates():
    joints = []
    weights = []
    inverse_bind_matrices = []
    for mesh in gltf.meshes:
        for primitive in mesh.primitives:
            joint_accessor = gltf.accessors[primitive.attributes.JOINTS_0]
            joint_buffer_view = gltf.bufferViews[joint_accessor.bufferView]
            # joint_buffer_data = gltf.buffers[joint_buffer_view.buffer].binary_blob
            #weights
            weight_accessor = gltf.accessors[primitive.attributes.WEIGHTS_0]
            weight_buffer_view = gltf.bufferViews[weight_accessor.bufferView]
            # weight_buffer_data = gltf.buffers[weight_buffer_view.buffer].binary_blob()
            for vertex_index in range(joint_accessor.count):
                
                joint_offset = joint_buffer_view.byteOffset + joint_accessor.byteOffset
                joints_for_vertex = np.frombuffer(
                    gltf.binary_blob(),
                    dtype=np.uint8,
                    count=4,
                    offset= joint_offset + vertex_index * 4 * np.dtype(np.uint8).itemsize
                )
                
                joints.append(joints_for_vertex)
                
                weight_offset = weight_buffer_view.byteOffset + weight_accessor.byteOffset
                weights_for_vertex = np.frombuffer(
                    gltf.binary_blob(),
                    dtype=np.float32,
                    count=4,
                    offset= weight_offset + vertex_index*4*np.dtype(np.float32).itemsize
                )
                weights.append(weights_for_vertex)
                for joint_index in joints_for_vertex:
                    inverse_bind_matrices.append(np.linalg.inv( get_joint_transform(gltf,joint_index)))
    joints = np.array(joints)  # Joint indices per vertex
    weights = np.array(weights)
    

    

    # Apply skinning transformation
    skinned_vertices = apply_skinning(gltf, vBind, joints, weights, inverse_bind_matrices)
    # Get the POSITION accessor
    position_accessor = gltf.accessors[mesh.primitives[0].attributes.POSITION]
    position_buffer_view = gltf.bufferViews[position_accessor.bufferView]

    # Offset within the buffer
    position_offset = position_buffer_view.byteOffset + position_accessor.byteOffset

    # Convert skinned vertices to binary
    skinned_vertex_data = skinned_vertices.astype(np.float32).tobytes()

    # Overwrite the binary blob with the skinned vertex positions
    gltf.buffers[position_buffer_view.buffer].data = (
        gltf.binary_blob()[:position_offset] + 
        skinned_vertex_data + 
        gltf.binary_blob()[position_offset + len(skinned_vertex_data):]
    )
    print(skinned_vertices.shape)
    gltf.save("3D/magicmirror.glb")
# vBind = []
# for v in vertices:
#     vBind.append(v.position)
# for v in vBind:
#     inverted = v * inverse_bind_matrix
#     transformed_joint = inverted*transformation_matrix
#     weightedSum = 0
#     for i in range(4):
#         transformed_joint*= v.gltf.accessors["WEIGHTS_0"]
#         weightedSum += transformed_joint
#     v.position = weightedSum
# Start the Pyrender viewer





# def active_update():
#     shirt_mesh = trimesh.load('3D\magicmirror.glb')
#     scene = pyrender.Scene()
#     scene.add(pyrender.Mesh.from_trimesh(trimesh.util.concatenate(shirt_mesh.dump())))
#     camera = pyrender.PerspectiveCamera(yfov=60, aspectRatio=1.0)
#     pose = np.eye(4)
#     pose[:3,3]=[0,0,2]
#     scene.add(camera, pose=pose)

#     renderer = pyrender.OffscreenRenderer(viewport_width=640,viewport_height=480)
#     color,_=renderer.render(scene)

#     color_bgr=cv2.cvtColor(color, cv2.COLOR_RGB2BGR)

#     cv2.imshow("gltf",color_bgr)
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         cv2.destroyAllWindows()



bone hip rotation:[9.777748346095905e-06, 0.0002718785544857383, -0.03592561557888985, 0.9993544816970825] and translation:[0.05732102319598198, -0.006602108478546143, 0.0]
bone Bone.002 rotation:[0.004490206018090248, 0.01330167893320322, 0.11290524154901505, 0.9935065507888794] and translation:[-3.725290298461914e-09, 1.0250910520553589, 0.0]
bone neck rotation:[-0.06400275230407715, -0.2615671157836914, -0.16527314484119415, 0.9487735033035278] and translation:[-2.922024577856064e-08, 1.621514081954956, -9.313225746154785e-10]
bone Bone.004 rotation:[-0.7730085253715515, -0.04162666201591492, 0.03303844854235649, 0.632165789604187] and translation:[-2.922024577856064e-08, 1.621514081954956, -9.313225746154785e-10]
bone leftShoulder rotation:[0.07886780798435211, 0.0008110007620416582, 0.04113754257559776, 0.996035635471344] and translation:[-1.1175870895385742e-07, 0.9313156604766846, 1.9185245037078857e-07]
bone leftForearm rotation:[0.008933612145483494, 0.12852653861045837, -0.03

# 1. Swipe Detection

In [5]:
def leftSwipeDetection(leftWristLocations):
    return leftWristLocations[0][0] - leftWristLocations[-1][0] > 100

def rightSwipeDetection(rightWristLocations):
    return rightWristLocations[-1][0] - rightWristLocations[0][0] > 100

def upSwipeDetection(rightWristLocations):
    return rightWristLocations[0][1] - rightWristLocations[-1][1] > 100


# Configure the logger
logging.basicConfig(
    filename='debug_log.txt',  # Output file
    level=logging.DEBUG,       # Logging level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Log message format
    filemode='w'  # Overwrite the log file each run (change to 'a' to append)
)

logger = logging.getLogger()


In [6]:

def mapTops(pose_points, img):
    # Calculate the distance between the shoulders (pose_points[0] and pose_points[1])
    shoulder_distance = math.sqrt((pose_points[1][0] - pose_points[0][0]) ** 2 + (pose_points[1][1] - pose_points[0][1]) ** 2)
    
    # Scale offsets based on shoulder distance
    shoulder_scale_factor = shoulder_distance / 250  # Adjust the denominator based on your needs; it's a scaling factor.
    shoulder_x_offset = int(100 * shoulder_scale_factor)
    hip_x_offset = int(180 * shoulder_scale_factor)
    shoulder_y_offset = int(50 * shoulder_scale_factor)
    
    # Calculate the height-to-width ratio of the input image
    heightToWidth = img.shape[0] / img.shape[1]
    width = int(math.fabs(pose_points[1][0] - pose_points[0][0]) * heightToWidth)
    height = int(width * heightToWidth)
    
    src = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype='float32')
    
    dest = np.array([
        [pose_points[0][0] + shoulder_x_offset, pose_points[0][1] - shoulder_y_offset], # Left shoulder
        [pose_points[1][0] - shoulder_x_offset, pose_points[1][1] - shoulder_y_offset], # Right shoulder
        [pose_points[3][0] - hip_x_offset, pose_points[3][1]], # Right hip
        [pose_points[2][0] + hip_x_offset, pose_points[2][1]] # Left hip
    ], dtype='float32')
    
    # Logging pose points and their mapping
    logger.debug(f"Og Pose Points: {pose_points}")
    logger.debug(f"Source Points (src): {src}")
    logger.debug(f"Destination Points (dest): {dest}")
    
    transform = cv2.getPerspectiveTransform(src, dest)
    img = cv2.resize(img, (width, height))
    
    return img, transform


def mapBottoms(pose_points, img):
    # Calculate the distance between the shoulders (pose_points[0] and pose_points[1])
    hip_distance = math.sqrt((pose_points[1][0] - pose_points[0][0]) ** 2 + (pose_points[1][1] - pose_points[0][1]) ** 2)
    
    # Scale offsets based on shoulder distance
    hip_scale_factor = hip_distance / 250  # Adjust the denominator based on your needs; it's a scaling factor.
    hip_x_offset = int(150 * hip_scale_factor) # 180 -> 150
    ankle_x_offset = int(150 * hip_scale_factor) # 200 -> 150
    # ankle_y_offset = int(200 * hip_scale_factor)
    hip_y_offset = int(70 * hip_scale_factor) # 10 -> 70
    
    # Calculate the height-to-width ratio of the input image
    heightToWidth = img.shape[0] / img.shape[1]
    width = int(math.fabs(pose_points[1][0] - pose_points[0][0]) * heightToWidth)
    height = int(width * heightToWidth)
    
    src = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype='float32')
    
    dest = np.array([
        [pose_points[0][0] + hip_x_offset, pose_points[0][1] - hip_y_offset], # Left shoulder
        [pose_points[1][0] - hip_x_offset, pose_points[1][1] - hip_y_offset], # Right shoulder
        [pose_points[3][0] - ankle_x_offset, pose_points[3][1]], # Right hip
        [pose_points[2][0] + ankle_x_offset, pose_points[2][1]] # Left hip
    ], dtype='float32')
    
    # Logging pose points and their mapping
    logger.debug(f"Og Pose Points: {pose_points}")
    logger.debug(f"Source Points (src): {src}")
    logger.debug(f"Destination Points (dest): {dest}")
    
    transform = cv2.getPerspectiveTransform(src, dest)
    img = cv2.resize(img, (width, height))
    
    return img, transform

def overlay_clothing(frame, clothing_img, transform):
    # Debugging: Shape of inputs before transformation
    logger.debug(f"Frame shape: {frame.shape}, Clothing image shape: {clothing_img.shape}")

    # Warp the clothing image to match the person's pose
    transformed_clothing = cv2.warpPerspective(clothing_img, transform, (frame.shape[1], frame.shape[0]))
    logger.debug(f"Transformed clothing image shape: {transformed_clothing.shape}")

    # Handle alpha blending if clothing image has an alpha channel
    if transformed_clothing.shape[-1] == 4:  # If the image has an alpha channel
        logger.debug("Clothing image has an alpha channel.")
        alpha_channel = transformed_clothing[:, :, 3] / 255.0  # Normalize alpha channel to 0-1
        for c in range(0, 3):  # Iterate over the RGB channels
            frame[:, :, c] = (1.0 - alpha_channel) * frame[:, :, c] + alpha_channel * transformed_clothing[:, :, c]
    else:
        logger.debug("Clothing image does not have an alpha channel.")
        # Directly overlay the transformed clothing (no transparency)
        mask = transformed_clothing > 0  # Consider any non-zero pixel as part of the clothing
        frame[mask] = transformed_clothing[mask]

    logger.debug("Overlay operation completed.")
    return frame

def capture_and_rotate_frame(camera, angle):
    ret, frame = camera.read()
    if not ret:
        return None

    # Rotate the frame according to the specified angle
    if angle == 90:
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    elif angle == -90:
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif angle == 180:
        frame = cv2.rotate(frame, cv2.ROTATE_180)

    return frame

In [7]:
leftWristLocations = deque(maxlen=10)
rightWristLocations = deque(maxlen=10)

In [ ]:
desktop = False

cap = cv2.VideoCapture(0)

angle = 0 if desktop else -90

i=0

last_left_swipe_time = 0
last_right_swipe_time = 0
swipe_delay = 1

clothing_tops = ["White T-shirt", "Blue T-shirt", "Red T-shirt"]
clothing_bottoms = ["Green Pants", "Tan Slacks", "White Pants"]

clothing_tops_index = 0
clothing_bottoms_index = 0
clothing_top_selected = False

## Setup mediapipe instanceq
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while True:
        # frame = capture_and_rotate_frame(cap, angle)
        
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame is None: 
            print("Frame capture returned None; ending loop")
            break
        
        #frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

        listTops = os.listdir("top_images")
        listBottoms = os.listdir("bottom_images")
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
            
                # Get coordinates
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                norm_coord = [frame.shape[0], frame.shape[1]] if desktop else [frame.shape[1], frame.shape[0]]

                # Normalizes Coordinates
                left_shoulder_coords = tuple(np.multiply(left_shoulder, norm_coord).astype(int))
                right_shoulder_coords = tuple(np.multiply(right_shoulder, norm_coord).astype(int))

                left_hip_coords = tuple(np.multiply(left_hip, norm_coord).astype(int))
                right_hip_coords = tuple(np.multiply(right_hip, norm_coord).astype(int)) 
                
                left_ankle_coords = tuple(np.multiply(left_ankle, norm_coord).astype(int))
                right_ankle_coords = tuple(np.multiply(right_ankle, norm_coord).astype(int))

                upper_coords = [left_shoulder_coords, right_shoulder_coords, left_hip_coords, right_hip_coords]
                lower_coords = [left_hip_coords, right_hip_coords, left_ankle_coords, right_ankle_coords]

                #testing***
                
               
                for bone in bone_list:
                    if bone.name == "rightShoulder":
                        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
                        # left_elbow_coords = tuple(np.multiply(left_elbow, norm_coord).item())
                        y = left_elbow[1]-left_shoulder[1]
                        x = left_elbow[0]-left_shoulder[0]
                        z = left_elbow[2]-landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z
                        d = math.sqrt(math.pow(y,2)+math.pow(x,2))
                        #find rotation angle of the bone
                        joint_node = pyrender.Node(name="rightShoulder", matrix = np.eye(4), rotation = [math.sin(math.asin(y/d)/2),0,math.sin(math.atan(x/z)/2),math.cos(math.asin(y/d)/2)] )
                        s.add_node(joint_node)
                        # bone.rotation = [math.sin(math.asin(y/d)/2),0,math.sin(math.atan(x/z)/2),math.cos(math.asin(y/d)/2)]
                        # bone.translation = [0.078867807894,left_shoulder[0],left_shoulder[1]]
                        
                for bone in bone_list:
                    if bone.name == "leftShoulder":
                        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]
                        # right_elbow_coords = tuple(np.multiply(right_elbow, norm_coord).item())
                        y = right_elbow[1]-right_shoulder[1]
                        x = right_elbow[0]-right_shoulder[0]
                        z = right_elbow[2]-landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z
                        d = math.sqrt(math.pow(y,2)+math.pow(x,2))
                        #find rotation angle of the bone
                        joint_node = pyrender.Node(name="leftShoulder", matrix = np.eye(4), rotation = [-math.sin(math.asin(y/d)/2),0,-math.sin(math.atan(x/z)/2),math.cos(math.asin(y/d)/2)] )
                        s.add_node(joint_node)
                        # bone.rotation = [-math.sin(math.asin(y/d)/2),0,-math.sin(math.atan(x/z)/2),math.cos(math.asin(y/d)/2)]
                        # bone.translation = [-0.117773614, right_shoulder[0],right_shoulder[1]]
                for bone in bone_list:
                    if bone.name == "hip":#fix hip rotation
                        z = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z-landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z
                        y = right_hip[1]-left_hip[1]
                        x = right_hip[0]-left_hip[0]
                        d = math.sqrt(math.pow(y,2)+math.pow(x,2))
                        if(x>=0):
                            joint_node = pyrender.Node(name="hip", matrix = np.eye(4), rotation = [0,math.sin(math.atan(x/z)/2),0,math.cos(math.asin(y/d)/2)] )
                            bone.rotation = [0,math.sin(math.atan(x/z)/2),0,math.cos(math.asin(y/d)/2)]
                        elif(x<0):
                            joint_node = pyrender.Node(name="hip", matrix = np.eye(4), rotation = [0,(1+math.sin(math.atan(x/z)/2)),0,1-math.cos(math.asin(y/d)/2)] )
                            # bone.rotation = [0,(1+math.sin(math.atan(x/z)/2)),0,1-math.cos(math.asin(y/d)/2)]
                        joint_node = pyrender.Node(name="hip", matrix = np.eye(4), translation = [(left_hip[0] + right_hip[0])/2,(left_hip[1] + right_hip[1])/2, (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z + landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z)/2] )
                        s.add_node(joint_node)

                        # bone.translation = [(left_hip[0] + right_hip[0])/2,(left_hip[1] + right_hip[1])/2, (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z + landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z)/2]
                bone.translation = [float(val) for val in bone.translation]
                bone.rotation = [float(val)/np.linalg.norm(bone.rotation) for val in bone.rotation]
                for node in s.nodes:
                    print(f"Node Name: {node.name}")
                    print(f"Node Translation: {node.translation}")
                    print(f"Node Rotation: {node.rotation}")

                try:
                    gltf.save_binary("3D/magicmirror.glb")
                    
                    # pose = np.eye(4)
                    # pose[:3,3] = [i,0,0]
                    # v.render_lock.acquire()
                    # scene.set_pose(node,pose)
                    # v.render_lock.release()
                    # i+=0.01
                    # active_update()
                    
                except Exception as e:
                    print(f"Error saving GLB file: {e}")

                #end of testing***

                for point in [left_shoulder_coords, right_shoulder_coords, left_hip_coords, right_hip_coords]:
                    cv2.circle(frame, point, 5, (0, 255, 0), -1)

                bottom_image = cv2.imread(os.path.join("bottom_images", listBottoms[clothing_bottoms_index]),cv2.IMREAD_UNCHANGED)
                bottom_image, transform = mapBottoms(lower_coords, bottom_image)
                frame = overlay_clothing(frame, bottom_image, transform)
                
                top_image = cv2.imread(os.path.join("top_images", listTops[clothing_tops_index]),cv2.IMREAD_UNCHANGED)
                top_image, transform = mapTops(upper_coords, top_image)
                frame = overlay_clothing(frame, top_image, transform)


                # cv2.imshow("Transformed shirt", top_image)
                # cv2.imshow("Transformed bottom", bottom_image)
                
                # cv2.imshow("Overlayed Clothing", frame)
                active_updates()
                
                v = pyrender.Viewer(s)
                
                    
                
                # cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
               
            else:
                print("No pose landmarks detected")
                continue
            
            # Normalizes Coordinates
            left_wrist_coords = tuple(np.multiply(left_wrist, [640, 480]).astype(int))
            right_wrist_coords = tuple(np.multiply(right_wrist, [640, 480]).astype(int))

            leftWristLocations.append(left_wrist_coords)
            rightWristLocations.append(right_wrist_coords)
            
            current_time = time.time()

            if leftSwipeDetection(leftWristLocations):
                if current_time - last_left_swipe_time > swipe_delay:
                    last_left_swipe_time = current_time
                    if clothing_top_selected:
                        if clothing_tops_index == 0:
                            clothing_tops_index = len(clothing_tops) - 1
                        else:
                            clothing_tops_index -= 1
                    else:
                        if clothing_bottoms_index == 0:
                            clothing_bottoms_index = len(clothing_bottoms) - 1
                        else:
                            clothing_bottoms_index -= 1
                        
                
            if rightSwipeDetection(rightWristLocations):
                if current_time - last_right_swipe_time > swipe_delay:
                    last_right_swipe_time = current_time
                    if clothing_top_selected:
                        if clothing_tops_index == len(clothing_tops) - 1 :
                            clothing_tops_index = 0
                            
                        else:
                            clothing_tops_index += 1
                    else:
                        if clothing_bottoms_index == len(clothing_bottoms) - 1:
                            clothing_bottoms_index = 0
                        else:
                            clothing_bottoms_index += 1
                        
           
            if upSwipeDetection(rightWristLocations):
                if current_time - last_right_swipe_time > swipe_delay:
                    last_right_swipe_time = current_time
                    clothing_top_selected = not clothing_top_selected
                    if clothing_top_selected: #if the boolean is false that means bottom array selected else top array
                        print("top array")
                    else:
                        print("bottom array")

            # swipeDetection(leftWristLocations, rightWristLocations, last_left_swipe_time, last_right_swipe_time, last_up_swipe_time, current_time, clothing_tops_index, clothing_bottoms_index, clothing_top_selected)

        except Exception as e:
            print(f"An error occurred during pose estimation or clothing mapping: {e}")
        except:
            pass
            
       
        # Draw a rectangle at the top of the screen for clothing item display
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        
        # Display clothing item name
        cv2.putText(image, 'CLOTHING ITEM', (15, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        if(clothing_top_selected):
            cv2.putText(image, clothing_tops[clothing_tops_index], 
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        else:
             cv2.putText(image, clothing_bottoms[clothing_tops_index], 
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
   

No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
No pose landmarks detected
Node Name: None
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: rightShoulder
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: hip
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: leftShoulder
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]


c:\Users\15rak\Documents\MagicMirror\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


(10471, 3)
Node Name: rightShoulder
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: hip
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: None
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: rightShoulder
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: leftShoulder
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: leftShoulder
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: hip
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
(10471, 3)
Node Name: rightShoulder
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: hip
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: None
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: rightShoulder
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: leftShoulder
Node Translation: [0. 0. 0.]
Node Rotation: [0. 0. 0. 1.]
Node Name: rightShoulder
Node Tra